In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
import random
import tqdm

In [2]:
template = {'food': 0, 'water':0, 'medical': 0}
limits = {'food': {'lower':2, 'upper':10}, 'water': {'lower':2, 'upper':10}, 'medical': {'lower':2, 'upper':10}}

In [30]:
class Distribution_Center(object):
    def __init__(self, dc_template=template, store_limit=100, limit = 1000, nstores = 10, s_template=template):
        self.stores = []
        self.limit = limit
        self.template = dc_template
        for k, v in dc_template.items():
            setattr(self, k, v)
            
        for idx, store in enumerate(range(nstores)):
                self.stores.append(Store(s_template, limit=store_limit))
            
    def change_attr(self, attr, rate):
        val = getattr(self, attr)
        newval = val + rate
        if newval < 0:
            newval = 0
        if newval > self.limit:
            newval = self.limit
        setattr(self, attr, newval)
        return(val-newval)
    
    def get_supply(self):
        ans = {}
        for val in self.template:
            ans[val] = getattr(self, val)
        return(ans)
            
    def reorder(self):
        for store in self.stores:
            for supply in store.template:
                currentval = getattr(store, supply)
                if currentval < store.re:
                    nextval = currentval + self.change_attr(supply, currentval - store.limit)
                    if nextval == 0:
                        print('not enough supply of {}'.format(supply))
                    setattr(store, supply, nextval)
                
        
    
            
    

In [31]:
class Store(Distribution_Center):
    def __init__(self, s_template=template, limit=100, re=20):
        self.template = s_template
        self.limit=limit
        self.re = re
        for k, v in s_template.items():
            setattr(self, k, v)
        super(Distribution_Center, self).__init__()
        
    def generate_customers_day(self, n):
        ans = []
        for val in range(n):
            ans.append(Customer())
        self.customers = ans
        
    def calc_demand(self):
        ans = {}
        for supply in self.template:
            ans[supply] = 0
        for customer in self.customers:
            for supply in self.template:
                ans[supply] += getattr(customer, supply)
        return ans
                
                
    def customer_shopping(self):
        for customer in self.customers:
            for supply in self.template:
                currentval = getattr(self, supply)
                self.change_attr(supply, -getattr(customer, supply))
    
        
        
    

In [32]:
class Customer(Distribution_Center):
    def __init__(self, limits=limits):
        for key, value in limits.items():
            setattr(self, key, random.choices(np.arange(value['lower'], value['upper']))[0])
            
        
            

In [33]:
aldi = Distribution_Center(dc_template={'food': 4000, 'water':4000, 'medical': 4000}, nstores=3, store_limit=700, limit=100000)
walmart = Distribution_Center(dc_template={'food': 4000, 'water':4000, 'medical': 4000}, nstores=5, store_limit=700, limit=100000)
dcs = [aldi, walmart]

def run_experiment(t=100):
    x = 634
    time = []
    dc_supply = {}
    store_supply = {}
    

    for supply in template:
        dc_supply[supply] = 0
        
    for i in tqdm.tqdm(range(t)):
        time.append(i)
        for idx, dc in enumerate(dcs):
            stock = dc.get_supply()
            
            for idx, stores in enumerate(dc.stores):
#             print('store {} has {} food {} water {} medical'.format(idx, stores.food, stores.water, stores.medical))
                stores.generate_customers_day(x)
                ans=stores.calc_demand()
            
                print('store {} has the following demand {} on day {}'.format(idx, str(ans), i))
                print('store {} has {} food {} water {} medical'.format(idx, stores.food, stores.water, stores.medical))
                stores.customer_shopping()
#             print('store {} has {} food {} water {} medical'.format(idx, stores.food, stores.water, stores.medical))
        
            print('\n')
        
            if aldi.change_attr('food', 1200) == 0:
                print('surplus of food')
            if aldi.change_attr('water', 1200) == 0:
                print('surplus of water')
            if aldi.change_attr('medical', 1200) == 0:
                print('surplus of medical')
            aldi.reorder()
            #x = int(x*random.uniform(1,2))
            print(x)
        
        
        
        

In [27]:
run_experiment(t=10)

 10%|████                                     | 1/10 [00:00<00:00,  9.63it/s]

store 0 has the following demand {'food': 3586, 'water': 3471, 'medical': 3489} on day 0
store 0 has 0 food 0 water 0 medical
store 1 has the following demand {'food': 3480, 'water': 3463, 'medical': 3530} on day 0
store 1 has 0 food 0 water 0 medical
store 2 has the following demand {'food': 3551, 'water': 3550, 'medical': 3522} on day 0
store 2 has 0 food 0 water 0 medical


634
store 0 has the following demand {'food': 3588, 'water': 3515, 'medical': 3484} on day 0
store 0 has 0 food 0 water 0 medical
store 1 has the following demand {'food': 3466, 'water': 3549, 'medical': 3515} on day 0
store 1 has 0 food 0 water 0 medical
store 2 has the following demand {'food': 3544, 'water': 3451, 'medical': 3388} on day 0
store 2 has 0 food 0 water 0 medical
store 3 has the following demand {'food': 3478, 'water': 3495, 'medical': 3420} on day 0
store 3 has 0 food 0 water 0 medical
store 4 has the following demand {'food': 3505, 'water': 3426, 'medical': 3506} on day 0
store 4 has 0 food 0 wa

 30%|████████████▎                            | 3/10 [00:00<00:00, 11.18it/s]


store 2 has 700 food 700 water 700 medical


634
store 0 has the following demand {'food': 3428, 'water': 3472, 'medical': 3567} on day 2
store 0 has 0 food 0 water 0 medical
store 1 has the following demand {'food': 3544, 'water': 3487, 'medical': 3454} on day 2
store 1 has 0 food 0 water 0 medical
store 2 has the following demand {'food': 3561, 'water': 3440, 'medical': 3445} on day 2
store 2 has 0 food 0 water 0 medical
store 3 has the following demand {'food': 3414, 'water': 3489, 'medical': 3400} on day 2
store 3 has 0 food 0 water 0 medical
store 4 has the following demand {'food': 3483, 'water': 3618, 'medical': 3489} on day 2
store 4 has 0 food 0 water 0 medical


634
store 0 has the following demand {'food': 3428, 'water': 3500, 'medical': 3440} on day 3
store 0 has 700 food 700 water 700 medical
store 1 has the following demand {'food': 3481, 'water': 3489, 'medical': 3530} on day 3
store 1 has 700 food 700 water 700 medical
store 2 has the following demand {'food': 3470, 'w

 70%|████████████████████████████▋            | 7/10 [00:00<00:00, 11.30it/s]

store 3 has the following demand {'food': 3469, 'water': 3455, 'medical': 3468} on day 4
store 3 has 0 food 0 water 0 medical
store 4 has the following demand {'food': 3539, 'water': 3383, 'medical': 3530} on day 4
store 4 has 0 food 0 water 0 medical


634
store 0 has the following demand {'food': 3505, 'water': 3478, 'medical': 3457} on day 5
store 0 has 700 food 700 water 700 medical
store 1 has the following demand {'food': 3539, 'water': 3455, 'medical': 3597} on day 5
store 1 has 700 food 700 water 700 medical
store 2 has the following demand {'food': 3513, 'water': 3384, 'medical': 3490} on day 5
store 2 has 700 food 700 water 700 medical


634
store 0 has the following demand {'food': 3496, 'water': 3529, 'medical': 3485} on day 5
store 0 has 0 food 0 water 0 medical
store 1 has the following demand {'food': 3472, 'water': 3531, 'medical': 3583} on day 5
store 1 has 0 food 0 water 0 medical
store 2 has the following demand {'food': 3388, 'water': 3528, 'medical': 3533} on day 5

 90%|████████████████████████████████████▉    | 9/10 [00:00<00:00, 11.37it/s]


store 2 has 700 food 700 water 700 medical


634
store 0 has the following demand {'food': 3489, 'water': 3426, 'medical': 3522} on day 7
store 0 has 0 food 0 water 0 medical
store 1 has the following demand {'food': 3539, 'water': 3483, 'medical': 3377} on day 7
store 1 has 0 food 0 water 0 medical
store 2 has the following demand {'food': 3433, 'water': 3453, 'medical': 3422} on day 7
store 2 has 0 food 0 water 0 medical
store 3 has the following demand {'food': 3479, 'water': 3447, 'medical': 3575} on day 7
store 3 has 0 food 0 water 0 medical
store 4 has the following demand {'food': 3584, 'water': 3506, 'medical': 3506} on day 7
store 4 has 0 food 0 water 0 medical


634
store 0 has the following demand {'food': 3485, 'water': 3561, 'medical': 3559} on day 8
store 0 has 700 food 700 water 700 medical
store 1 has the following demand {'food': 3553, 'water': 3505, 'medical': 3510} on day 8
store 1 has 700 food 700 water 700 medical
store 2 has the following demand {'food': 3495, 'w

100%|████████████████████████████████████████| 10/10 [00:00<00:00, 11.36it/s]


store 3 has 0 food 0 water 0 medical
store 4 has the following demand {'food': 3534, 'water': 3571, 'medical': 3404} on day 9
store 4 has 0 food 0 water 0 medical


634


In [893]:
print('aldi has {} food {} water {} medical'.format(aldi.food, aldi.water, aldi.medical))

print('\n')

for idx, stores in enumerate(aldi.stores):
    print('store {} has {} food {} water {} medical'.format(idx, stores.food, stores.water, stores.medical))

aldi has 0 food 0 water 0 medical


store 0 has 100 food 100 water 100 medical
store 1 has 70 food 70 water 70 medical
store 2 has 100 food 100 water 100 medical
store 3 has 70 food 70 water 70 medical
store 4 has 100 food 100 water 100 medical
store 5 has 70 food 70 water 70 medical
store 6 has 0 food 0 water 0 medical
store 7 has 70 food 70 water 70 medical
store 8 has 0 food 0 water 0 medical
store 9 has 70 food 70 water 70 medical


In [39]:
walmart.stores[0]get_supply()

{'food': 4000, 'water': 4000, 'medical': 4000}